# Share Credential Account Management

In [ ]:
import os, json
import pandas as pd
import requests, urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import time

from dotenv import load_dotenv
load_dotenv()

from insightvm_authen import InsightvmAuthentication

In [ ]:
HOSTNAME = "https://" + os.getenv("HOSTNAME") + "/"

INSIGHTVM_USER = os.getenv("INSIGHTVM_USER")
INSIGHTVM_PASSWORD = os.getenv("INSIGHTVM_PASSWORD")

MAIN_HEADER = InsightvmAuthentication().getRequestHeader()
MAIN_HEADER

## 0. Create InsightVM Session

In [ ]:
# Authentication
loginURL = HOSTNAME + 'data/user/login'
postData = {'nexposeccusername': INSIGHTVM_USER, 'nexposeccpassword': INSIGHTVM_PASSWORD}

print("\nLogging into Nexpose (" + loginURL + ")")
r = requests.post(loginURL, data=postData, verify=False)
cookie = dict(nexposeCCSessionID=str(r.cookies['nexposeCCSessionID']))
print("\nAuthentication Successful")
cookie

## 1. Create Share Credential Account

### File Input Pattern Format

General Pattern

```
credential_name,service,appid,cyberark_username,cyberark_address,cyberark_domain,restrictUrl,os
```

Example for CIFS

```
Test Cyberark Script Samba,cifs,Rapid7test,rapid7usr,XXX.com,XXX.com,test.XXX.com,
```

Example for SSH

```
Test Cyberark Script SSH,ssh,Rapid7,rapid7usr,test.XXX.com,,test.XXX.com,redhat_aws
```

In [ ]:
def createSharedCredentialCyberark(name="", service="", appid="", username="", address="", domain="", escalation=["", "", ""], allsite="0", restrictUrl=""):
    cyberark_safe = "Safe"
    cyberark_folder = "root"

    createSharedCredentialURL = HOSTNAME + 'data/credential/shared/save'
    payloads = '<Credential shared="1" id="-1" enabled="0"><Name>' + name + '</Name><Description></Description><Services><Service type="' + service + '"/></Services><Account type="cyberark"><Field name="database"></Field><Field name="domain">' + \
        domain + '</Field><Field name="username"></Field><Field name="ntlmhash"></Field><Field name="password"></Field><Field name="pemkey"></Field><Field name="privilegeelevationusername">' + escalation[1] + '</Field><Field name="privilegeelevationpassword">' + \
        escalation[2] + '</Field><Field name="snmpv3privpassword"></Field><Field name="privilegeelevationtype">' + escalation[0] + '</Field><Field name="appid">' + appid + '</Field><Field name="lookuptype">static</Field><Field name="safe">' + cyberark_safe + '</Field>' + \
        '<Field name="folder">' + cyberark_folder + '</Field><Field name="searchall">false</Field><Field name="address">' + address + '</Field><Field name="cyberarkusername">' + username + '</Field><Field name="realm"></Field></Account>' + \
        '<Restrictions><Restriction type="host">' + restrictUrl + '</Restriction><Restriction type="port"></Restriction></Restrictions><Sites all="' + allsite + '"/></Credential>'
    # print(payloads)

    headers = {
        'Content-Type': 'text/xml;charset=UTF-8',
        "Nexposeccsessionid": str(cookie['nexposeCCSessionID'])
    }

    print("\nCreate Shared Credential")
    print("Hostname : " + createSharedCredentialURL)

    r = requests.post(createSharedCredentialURL, data=payloads, headers=headers, cookies=cookie, verify=False)
    if r.status_code == 200 and str(r.text).strip() == "<ajaxResponse success=\"1\"></ajaxResponse>":
        print("[Success] Creating Successful " + name)
        return
    else:
        print("[Error] Request Connection Error.")
        print(r.status_code)
        print(r.text)


In [ ]:
# API Version - Not cover requirement
#
# def createSharedCredentialInsighVM(self, service="", domain="", ip=""):
#     createSharedCredentialURL = self.HOSTNAME + 'api/3/shared_credentials'
#     payloads = {
#         "name": "Test add shared credential",
#         "description": "This is a cifs test credential",
#         "account": {
#             "service": service,
#                 "domain": domain,
#                 "username": "test",
#                 "password": "test",
#         },
#         "siteAssignment": "all-sites",
#         # "siteAssignment": "specific-sites", "sites": [],
#         "hostRestriction": ip
#     }
#     print(payloads)
#     print("\nCreate Shared Credential")
#     print("Hostname : " + createSharedCredentialURL)

#     r = requests.post(createSharedCredentialURL, data=json.dumps(payloads).encode("utf-8"),
#                       headers=self.HEADER, verify=False)
#     if r.status_code == 201:
#         r = r.json()
#         if r['id']:
#             print("Your Report Configuration ID : " + str(r['id']))
#             return str(r['id'])
#         else:
#             print("[Error] Can't create Shared Credential")
#     else:
#         print("[Error] Request Connection Error.")
#         print(r.status_code)


In [ ]:
def generateEscalation(os=""):
    if os in ["redhat", "redhat_aws"]:
        return ["SUDOSU", "root", ""]
    else:
        return ["NONE", "", ""]

In [ ]:
filepath = "inputs/input_cred.csv"

In [ ]:
with open(filepath, "r") as datas:
    for index, line in enumerate(datas):
        if index > 0 and line != "":
            cred = line.strip().split(",")
            createSharedCredentialCyberark(name=cred[0], service=cred[1], appid=cred[2], username=cred[3], address=cred[4], domain=cred[5], escalation=generateEscalation(os=cred[7]), allsite="1", restrictUrl=cred[6])
            time.sleep(2)
            if index > 0 and index % 5 == 0:
                time.sleep(10)

## 2. List Share Credential Account

In [ ]:
def getSharedCredentials():
    print("\nGetting Shared Credential")
    r = requests.get(HOSTNAME + "api/3/shared_credentials", headers=MAIN_HEADER, verify=False)
    if r.status_code == 200:
        r = r.json()
        if r["resources"]:
            return {"data": r["resources"], "total": len(r["resources"])}
        else:
            print("\n[Not Found] Your Shared Credentials")
    else:
        print("[Error] Request Connection Error.")
        print(r.status_code)

In [ ]:
sharedCredentials = getSharedCredentials()

In [ ]:
sharedCredentials_df = pd.DataFrame(sharedCredentials["data"])
sharedCredentials_df.sort_values(by=['id'])

In [ ]:
filepath="outputs/sharedCredentialaccounts.json"

In [ ]:
with open(filepath, 'w', encoding="utf8") as f:
    f.write(json.dumps(getSharedCredentials, indent=4))
print("\n[Success Exporting] " + filepath)

In [ ]:
# getSharedCredentials_df[getSharedCredentials_df["name"]=="cyberark_XXX.com"]["id"][0]

## 3. Remove Share Credential Account

In [ ]:
def deleteSharedCredential(sharedCredentialID=""):
    print("\nGetting Shared Credential")
    r = requests.delete(HOSTNAME + "api/3/shared_credentials/" + str(sharedCredentialID), headers=MAIN_HEADER, verify=False)
    if r.status_code == 200:
        print("[Success] Delete Shared Credentia ID %s Successful" % str(sharedCredentialID))
        return
    else:
        print("[Error] Request Connection Error.")
        print(r.status_code)
        time.sleep(10)

In [ ]:
startID = 1
stopID = 1000

In [ ]:
for i in range(startID, stopID+1, 1):
    deleteSharedCredential(sharedCredentialID=str(i))

### Delete Exist Credential from API Lab

In [ ]:
filepath = "inputs/sharedCredential_all.csv"
credentialId_all = []

sharedCredentials = getSharedCredentials()
sharedCredentials_df = pd.DataFrame(sharedCredentials["data"])

with open(filepath, "r") as datas:
    for index, line in enumerate(datas):
        if index > 0 and line != "":
            cred = line.strip().split(",")
            print(cred)
            credentialId = sharedCredentials_df[sharedCredentials_df["name"]==cred[0]]["id"]
            if not credentialId.empty:
                credentialId_all.append(credentialId.values[0])
sorted(set(credentialId_all))

In [ ]:
for index, credentialId in enumerate(sorted(set(credentialId_all))):
    deleteSharedCredential(sharedCredentialID=str(credentialId))
    time.sleep(2)
    if index > 0 and index % 5 == 0:
        time.sleep(10)